In [4]:
from imblearn.combine import SMOTEENN
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, plot_confusion_matrix
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn import tree
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

from sklearn.svm import SVC

from sklearn.metrics import classification_report
from collections import Counter
from imblearn.under_sampling import EditedNearestNeighbours
import pydotplus
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.linear_model import  LogisticRegression
from sklearn.tree import DecisionTreeClassifier

from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler

In [5]:
df = pd.read_csv("diabetes.csv")
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


1.	Perform combined over and undersampling on the diabetes dataset (use SMOTEENN). Explain how combined sampling works.

Oversampling - creates new synthetic examples in a minority class you're trying to make predictions about (more people with diabetes)

Undersampling - delete or merge examples in a majority class

Combined is both at the same time.  

In [6]:
df['Outcome'].value_counts()


0    500
1    268
Name: Outcome, dtype: int64

In [7]:
X = df.drop('Outcome',axis = 1)
y = df['Outcome']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 27, stratify = y)


In [8]:
sc = StandardScaler()

X_train_sc = sc.fit_transform(X_train)
X_test_sc = sc.fit_transform(X_test)

In [9]:
lr = LogisticRegression()
lr.fit(X_train_sc,y_train)
y_pred = lr.predict(X_test_sc)

accuracy_score=accuracy_score(y_test,y_pred)
print("No SMOTE-ENN :",accuracy_score)

No SMOTE-ENN : 0.7922077922077922


In [10]:
print(classification_report(y_pred,y_test))


              precision    recall  f1-score   support

           0       0.89      0.81      0.85       166
           1       0.60      0.75      0.67        65

    accuracy                           0.79       231
   macro avg       0.75      0.78      0.76       231
weighted avg       0.81      0.79      0.80       231



In [11]:
#do with SMOTE ENN

X_smoteenn = df.drop('Outcome',axis = 1)
y_smoteenn = df['Outcome']

X_train, X_test, y_train, y_test = train_test_split(X_smoteenn,
                                                    y_smoteenn, test_size = 0.3, random_state = 42, stratify = y)



In [12]:

sc = StandardScaler()
X_train_sc = sc.fit_transform(X_train)
X_test_sc = sc.fit_transform(X_test)

In [13]:
sme = SMOTEENN(random_state=27,enn=EditedNearestNeighbours(sampling_strategy='all'))
X_resample, y_resample = sme.fit_resample(X_train_sc, y_train)

In [14]:
lr2 = LogisticRegression(random_state=42)
lr2.fit(X_resample, y_resample)

LogisticRegression(random_state=42)

In [15]:
y_pred = lr2.predict(X_test_sc)
balanced_accuracy_score(y_test, y_pred)

0.770246913580247

In [16]:
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

           0       0.71      0.88      0.79       121
           1       0.83      0.61      0.70       110

    accuracy                           0.75       231
   macro avg       0.77      0.75      0.75       231
weighted avg       0.77      0.75      0.75       231



2.	Comment on the performance of combined sampling vs the other approaches we have used for the diabetes dataset.

 my smote-enn performance seems to be worse than the non smote-enn version. The Logistic Regression appears to be the best if we are going off of Recall. 

In [42]:
X = df.drop('Outcome',axis = 1)
y = df['Outcome']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 27, stratify = y)


#Instantiate individual classifiers
lr = LogisticRegression(random_state = 27, max_iter = 7500)
knn = KNN()
dt = DecisionTreeClassifier(random_state = 27)

# definie a list called classifier that contains the tuples 
#(classifier_name, classifier)
classifiers = [('Logistic Regression', lr), 
               ('K Nearest Neighbors', knn), 
              ('Classification Tree', dt)]

for clf_name, clf in classifiers: 
    #fit clf to the training set 
    clf.fit(X_train, y_train)
    
    #predict the labels of the test set 
    y_pred = clf.predict(X_test)
    
    #evaluate the accuracy of clf on the test set 
    #print(accuracy_score(y_test, y_pred))
    print ('{:s} : {:.3f}'.format(clf_name +  ' accuracy_score ', balanced_accuracy_score(y_test, y_pred)))
    print ('{:s} : {:.3f}'.format(clf_name + ' recall', recall_score(y_test, y_pred)))
    print ('{:s} : {:.3f}'.format(clf_name + ' precision', precision_score(y_test, y_pred)))
    print ((clf_name + ' classification report'))
    print(classification_report(y_pred,y_test))    
    print("\n")
    


Logistic Regression accuracy_score  : 0.755
Logistic Regression recall : 0.630
Logistic Regression precision : 0.739
Logistic Regression classification report
              precision    recall  f1-score   support

           0       0.88      0.81      0.85       162
           1       0.63      0.74      0.68        69

    accuracy                           0.79       231
   macro avg       0.75      0.78      0.76       231
weighted avg       0.81      0.79      0.80       231



K Nearest Neighbors accuracy_score  : 0.670
K Nearest Neighbors recall : 0.593
K Nearest Neighbors precision : 0.558
K Nearest Neighbors classification report
              precision    recall  f1-score   support

           0       0.75      0.77      0.76       145
           1       0.59      0.56      0.57        86

    accuracy                           0.69       231
   macro avg       0.67      0.67      0.67       231
weighted avg       0.69      0.69      0.69       231



Classification Tree accu

3.	What is outlier detection? Why is it useful? What methods can you use for outlier detection?


Outlier detection is any way you're identifying outliers or oddballs in the data. Removing helps reduce overfitting. My favorite method of detection is the boxplot since I like a nice visual. You can easily see which points are outliers. 
Other techniques are Minimum Covariance Determinant, Local Outlier Factor, iForest and more. 

4.	Perform a linear SVM to predict credit approval (last column) using this dataset: https://archive.ics.uci.edu/ml/datasets/Statlog+%28Australian+Credit+Approval%29 . Make sure you look at the accompanying document that describes the data in the dat file. You will need to either convert this data to another file type or import the dat file to python. 
You can use this code, but otherwise you follow standard practices we have already used many times: 
from sklearn.svm import SVC
classifier = SVC(kernel='linear')


In [19]:

credit_df = pd.read_table("australian.dat", sep=" ",header=None)
credit_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,1,22.08,11.46,2,4,4,1.585,0,0,0,1,2,100,1213,0
1,0,22.67,7.00,2,8,4,0.165,0,0,0,0,2,160,1,0
2,0,29.58,1.75,1,4,4,1.250,0,0,0,1,2,280,1,0
3,0,21.67,11.50,1,5,3,0.000,1,1,11,1,2,0,1,1
4,1,20.17,8.17,2,6,4,1.960,1,1,14,0,2,60,159,1


In [ ]:
X = credit_df.drop(14,axis = 1)
y = credit_df[14]

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 27)

In [ ]:
sc = StandardScaler()
X_train_scaler = sc.fit_transform(X_train)
X_test_scaler = sc.fit_transform(X_test)

In [ ]:
classifier = SVC(kernel='linear')

#fit the model to the dataset on the training data
classifier.fit(X_train_scaler, y_train)
print(classifier.score(X_train_scaler,y_train))
print(classifier.score(X_test_scaler,y_test))

#make predictions using the scaled test dataset
y_pred = classifier.predict(X_test_scaler)



5.	How did the SVM model perform? Use a classification report. 

In [179]:
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.84      0.90      0.87        86
           1       0.89      0.83      0.86        87

    accuracy                           0.86       173
   macro avg       0.86      0.86      0.86       173
weighted avg       0.86      0.86      0.86       173



It looks like it did fairly well. It could absolutely be better, but it wasn't bad. 

6.	What kinds of jobs in data are you most interested in? Do some research on what is out there. Write about your thoughts in under 400 words. 

My dream would probably be to working with Data in the CIA. Ever since I was a kid I wanted to work for the CIA but never in the field, I always wanted to be in gathering intelligence. I think my days of hoping that could be a possibility are over, I have no desire to move to Virginia and I'm guessing there isn't a lot of remote work for CIA jobs. 
So far, based on what I've seen in this class, I'm leaning more towards data analysis than data science. It is a fascinating world but I don't feel as strong in it as I would want to be if I'm reaching out for a job in it. 

I'm actually coming back after week 20 to say that the language aspect is incredibly intriguing. I don't know how good I'll be at it yet since we just had class, but it seems amazing. 